<a href="https://colab.research.google.com/github/WiInfFelix/DLLSProjekt/blob/master/DLLS_Keras_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tqdm import tqdm
from spacy.lang.en import English
from pprint import pprint

import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.callbacks import EarlyStopping


from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
import spacy
import re
import time

In [0]:
pickle_list = "/content/drive/My Drive/Master Wirtschaftsinformatik/3. Semester /DLLS/DLLSProjekt/list.p"
pickle_texts = "/content/drive/My Drive/Master Wirtschaftsinformatik/3. Semester /DLLS/DLLSProjekt/texts.p"
dev_pickle = "/content/drive/My Drive/Master Wirtschaftsinformatik/3. Semester /DLLS/DLLSProjekt/dev_articles_pickle"
dev_articles = "/content/drive/My Drive/Master Wirtschaftsinformatik/3. Semester /DLLS/DLLSProjekt/datasets/dev-articles"
train_articles = ""

dataset_folder = "/content/drive/My Drive/Master Wirtschaftsinformatik/3. Semester /DLLS/DLLSProjekt/datasets"
train_texts = "/content/drive/My Drive/Master Wirtschaftsinformatik/3. Semester /DLLS/DLLSProjekt/datasets/train-articles"
train_labels = "/content/drive/My Drive/Master Wirtschaftsinformatik/3. Semester /DLLS/DLLSProjekt/datasets/train-labels-task1-span-identification"
model_save_file = "/content/drive/My Drive/Master Wirtschaftsinformatik/3. Semester /DLLS/DLLSProjekt/keras_model2.h5"


left_context = 4
right_context = 0

In [0]:
class Article:
    def __init__(self, article_id, tokens, words):
        self.tokens = tokens
        self.article_id = article_id
        self.words = words

class Word:
  def __init__(self, text, idx, prop_class):
    self.text = text
    self.idx = idx
    self.prop_class = prop_class

In [0]:
def get_data(train_test_size):
    input_list = pickle.load(open(pickle_list, "rb"))
    ## Take a subset
    corpus_words = [x[0] for x in input_list]
    corpus_tags = [x[1] for x in input_list]

    word_encoder = LabelEncoder()
    label_encoder = LabelEncoder()

    corpus_words_num = word_encoder.fit_transform(corpus_words)
    corpus_tags_num = label_encoder.fit_transform(corpus_tags)

    input_dim = len(word_encoder.classes_)
    output_dim = len(label_encoder.classes_)

    return word_encoder, label_encoder, corpus_words_num, corpus_tags_num, input_dim, output_dim


def prepare_data(left_context_len, right_context_len, train_test_size):
    word_encoder, label_encoder, x_data, y_data, input_dim, output_dim, = get_data(train_test_size)

    train_data = [(x_data[i - left_context_len:i + right_context_len + 1], y_data[i]) for i in
                  range(left_context_len, len(x_data) - right_context_len)]
    x_train = np.array([pair[0] for pair in train_data])
    y_train = np.array([pair[1] for pair in train_data])

    return input_dim, output_dim, x_train, y_train, word_encoder, label_encoder


def get_dev_data():
  nlp = spacy.load("en_core_web_sm")

  id_getter = re.compile(r'\d+')

  articles_dict = {}
  word_list = []

  num_file = len(os.listdir(dev_articles))
  print(f'{num_file} files have been found!')

  for entry in tqdm(os.scandir(dev_articles), total=75):
    with open(entry, encoding="utf-8") as file:
      text = file.read()
      tokens = nlp(text)
      np_tokens = np.array(tokens)
      #print(type(np_tokens))
      for tok in tokens:
        word_list.append(tok.text)
      #time.sleep(1)
      article_id = id_getter.search(entry.name).group(0)
      #article = Article(article_id, tokens)
      articles_dict[article_id] = np_tokens

  print(f'Finished scanning articles. {len(articles_dict)} have been read in...')

  print(f'Finished reading in!')

  return articles_dict, word_list



def main():
    # model size parameters
    left_context_len = 4
    right_context_len = 0

    # set this higher to get a better model
    train_test_size = 200000
    embedding_dim = 1000

    ## Hyperparemeters: experiment with these, too
    learning_rate = 0.01
    epochs =  10

    seq_len = left_context_len + 1 + right_context_len
    input_dim, output_dim, x_data, y_data, word_encoder, label_encoder = prepare_data(left_context_len, right_context_len, train_test_size)
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)
    #x, y, optimizer, loss, pred_argmax = build_graph(seq_len, input_dim, output_dim, embedding_dim, learning_rate)

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 4} ) 
    sess = tf.Session(config=config) 
    keras.backend.set_session(sess)

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
  
    model = Sequential()
    model.add(Embedding(input_dim, output_dim=int(input_dim**0.5)))
    model.add(LSTM(1024))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])
              #ä
    
    model = load_model(model_save_file)

    #model.fit(x_train, y_train, batch_size=32, epochs=20, validation_split=0.2, callbacks = [es])
    #model.save(model_save_file)  # creates a HDF5 file 'my_model.h5'
    #score = model.evaluate(x_test, y_test, batch_size=32)
    #print(score)

    dev_data, dev_list = get_dev_data()
    #print(dev_data)
    encoded_dev = {}
    dev_list_dec = dev_list

    '''
    for entry in dev_data:
      encoded_dev[entry.key()] = word_encoder.fit_transform(entry)

    for entry in encoded_dev:
      model.predict(entry.value())
    '''

    dev_list = word_encoder.fit_transform(dev_list)
    #dev_list = einteilen in ein Aufteilung wie train array

    dev_list = np.array(dev_list)

    res = model.predict(dev_list)

    first_res = (["o" if x >= 0.5 else "PROP" for x in res[:400]])

    print("{:15}||{}".format("Word", "Prediction"))
    print(30 * "=")
    for w, pred in zip(dev_list_dec[:400], first_res):
        print("{:15}: {:5}".format(w, pred))


if __name__ == '__main__':
    main()

In [0]:
Word           ||Prediction
==============================
From           : o    
Bad            : o    
To             : PROP 
Worse          : o    
?              : o    

              : o    
Tillerson      : o    
Out            : o    
–              : o    
CIA            : o    
Director       : o    
Pompeo         : o    
In             : o    
At             : o    
State          : PROP 
Dept           : o    
–              : o    
Gina           : o    
Haspel         : o    
As             : o    
Head           : o    
Of             : o    
CIA            : o    


             : o    
From           : o    
Bad            : o    
To             : PROP 
Worse          : o    
?              : o    

              : o    
Tillerson      : o    
Out            : o    
–              : o    
CIA            : o    
Director       : o    
Pompeo         : o    
In             : o    
At             : o    
State          : PROP 
Dept           : o    
–              : o    
Gina           : o    
Haspel         : o    
As             : o    
Head           : o    
Of             : o    
CIA            : o    

              : o    
Please         : o    
help           : PROP 
support        : o    
us             : o    
with           : o    
cryptocurrency : o    
donations      : o    
.              : o    

              : o    
Thank          : PROP 
you            : o    
!              : o    

              : o    
Keeping        : o    
things         : o    
moving         : o    
,              : o    
and            : o    
I              : o    
think          : o    
a              : o    
lot            : o    
of             : o    
us             : o    
can            : o    
just           : PROP 
shout          : o    
out            : o    
,              : o    
“              : o    
What           : o    
are            : o    
you            : o    
thinking       : o    
?              : o    
”              : o    
as             : o    
President      : o    
Donald         : PROP 
Trump          : o    
brings         : PROP 
CIA            : o    
Director       : o    
Mike           : o    
Pompeo         : o    
,              : o    
a              : o    
man            : o    
that           : PROP 
blasted        : o    
the            : o    
likes          : o    
of             : o    
Edward         : o    
Snowden        : o    
for            : o    
revealing      : o    
the            : o    
crimes         : o    
of             : o    
our            : o    
government     : o    
,              : o    
to             : o    
the            : o    
State          : PROP 
Department     : o    
and            : o    
installs       : o    
Gina           : o    
Haspel         : o    
in             : o    
the            : o    
position       : o    
of             : o    
head           : PROP 
of             : o    
the            : o    
Central        : PROP 
Intelligence   : o    
Agency         : o    
.              : o    

              : o    
Trump          : o    
tweeted        : o    
out            : o    
the            : o    
news           : o    
on             : o    
Tuesday        : PROP 
.              : o    

              : o    
“              : o    
Mike           : o    
Pompeo         : o    
,              : o    
Director       : o    
of             : o    
the            : o    
CIA            : o    
,              : o    
will           : o    
become         : PROP 
our            : o    
new            : o    
Secretary      : PROP 
of             : o    
State          : PROP 
.              : o    

              : o    
He             : o    
will           : o    
do             : PROP 
a              : o    
fantastic      : o    
job            : o    
!              : o    

              : o    
Thank          : PROP 
you            : o    
to             : o    
Rex            : PROP 
Tillerson      : o    
for            : o    
his            : o    
service        : o    
!              : o    

              : o    
Gina           : o    
Haspel         : o    
will           : o    
become         : PROP 
the            : o    
new            : o    
Director       : o    
of             : o    
the            : o    
CIA            : o    
,              : o    
and            : o    
the            : o    
first          : o    
woman          : o    
so             : PROP 
chosen         : PROP 
.              : o    

              : o    
Congratulations: PROP 
to             : o    
all            : o    
!              : o    
”              : o    
he             : o    
tweeted        : o    
.              : o    

              : o    
Mike           : o    
Pompeo         : o    
,              : o    
Director       : o    
of             : o    
the            : o    
CIA            : o    
,              : o    
will           : o    
become         : PROP 
our            : o    
new            : o    
Secretary      : PROP 
of             : o    
State          : PROP 
.              : o    

              : o    
He             : o    
will           : o    
do             : PROP 
a              : o    
fantastic      : o    
job            : o    
!              : o    

              : o    
Thank          : PROP 
you            : o    
to             : o    
Rex            : PROP 
Tillerson      : o    
for            : o    
his            : o    
service        : o    
!              : o    

              : o    
Gina           : o    
Haspel         : o    
will           : o    
become         : PROP 
the            : o    
new            : o    
Director       : o    
of             : o    
the            : o    
CIA            : o    
,              : o    
and            : o    
the            : o    
first          : o    
woman          : o    
so             : PROP 
chosen         : PROP 
.              : o    

              : o    
Congratulations: PROP 
to             : o    
all            : o    
!              : o    

              : o    
—              : o    
Donald         : PROP 
J.             : PROP 
Trump          : o    
(              : o    
@realDonaldTrump: o    
)              : PROP 
March          : PROP 
13             : o    
,              : o    
2018           : o    

              : o    
The            : PROP 
Washington     : o    
Post           : o    
first          : o    
reported       : o    
on             : o    
the            : o    
story          : o    
prior          : o    
to             : o    
Trump          : o    
’s             : o    
tweet          : o    
.              : o    

              : o    
Trump          : o    
and            : o    
Tillerson      : o    
have           : o    
had            : PROP 
a              : o    
fraught        : o    
relationship   : o    
for            : o    
many           : PROP 
months         : o    
.              : o    

              : o    
Trump          : o    
told           : o    
reporters      : o    
Tuesday        : PROP 
that           : PROP 
he             : o    
ultimately     : o    
decided        : o    
to             : o    
fire           : o    
the            : o    
secretary      : o    
because        : o    
they           : o    
disagreed      : o    
over           : o    
strategy       : o    
in             : o    
key            : o    
areas          : o    
of             : o    
foreign        : PROP 
policy         : PROP 
,              : o    
such           : o    
as             : o    
the            : o    
2015           : o    
Iran           : o    
nuclear        : o    
deal           : PROP 
,              : o    
the            : o    
approach       : o    
to             : o    
North          : o    
Korea          : o    
and            : o    
the            : o    
overall        : o    
tone           : o    
of             : o    
U.S.           : o    
diplomacy      : o    
.              : o    

              : o    
Tillerson      : o    
said           : o    
he             : o    
received       : o    
a              : o    
call           : o    
from           : o    
Trump          : o    
around         : o    
noon           : o    
Tuesday        : PROP 
,              : o    
more           : o    
than           : o    
three          : o    
hours          : o    
after          : PROP 
his            : o    
firing         : o    
was            : o    
first          : o    
reported       : o    
by             : o    
The            : PROP 
Washington     : o    
Post           : o    
and            : o    
announced      : o    
minutes        : PROP 
later          : PROP 
in             : o    
a              : o    
tweet          : o    
from           : o    
the            : o    
president      : o    
.              : o    

              : o    
His            : o    
voice          : o    
quivering      : o    
,              : o    
Tillerson      : o    
thanked        : o    
career         : o    
diplomats      : o    
for            : o    
their          : PROP 
“              : o    
honesty        : o    
and            : o    